- __Volumes__ are the mothods by which you attach storage to a pod

- some Volumes are ephemeral

- some Volumes are persistent

_Regardless of type, volumes are attached to a pod, not containers, If a pod isn't mapped to a node anymore, the volume isn't either._

Persistent storage options

- Are block storage, or networked file systems. _On GKE, these volumes are typically backed by Compute Engine persistent disks._

- Provide durable storage outside of Pod

- Are independent of the Pod's lifecycle

- May exist before Pod creation and be claimed

Volumes allow containers withi a Pod to share data

- Long-lived, persistent disk
- Short-lived, emptyDir
- Networked, NFS

__emptyDir__: ephemeral, share Pod's lifecycle  
__ConfigMap__: Object can be referenced in a volume, _set webserver's parameters_  
__Secret__: Stores sensitive info, such as passwords, _backed by in-memory file systems_  
__downwardAPI__: Makes data about Pods available to containers, _contaiener learn Pod's_

__example__

```
apiVersion: v1
kind: Pod
matadata:
    name: web
spec:
    containers:
    - name: web
      image: nginx
      volumeMounts:
      - mountPath: /mnt/vol
        name: nfs
    volumes:
    - name: nfs
      server: 10.1.1.2
      path: "/"
      readOnly: false
```

__Compute engine persistent disk__

```
gcloud compute disks create --size=100GB --zone=us-central1-a demo-disk
```

```
apiVersion: v1
kind: Pod
matadata:
    name: web
spec:
    containers:
    - name: web
      image: nginx
      volumeMounts:
      - mountPath: /mnt/vol
        name: pd-volume
    volumes:
    - name: pd-volume
      gcePersistentDisk:
          pdName: demo-disk
          fsType: ext4
```

__PersistentVolume__ independent of a Pod's lifecycle

__PersistentVolume(PV)__ 

- Independent of a Pod's lifecycle

- Managed by kubernets

- manually or dynamically provisioned

- persistent disks are used by GKE as PersistentVolumes

```
apiVersion: v1
kind: PersistentVolume
matadata:
    name: pd-volume
spec:
    storageClassName: "standard" #ssd
    capacity:
        storage: 100G
    accessModes:
    - ReadWriteOnce
    gcePersistentDisk:
        pdName: demo-disk
        fsType: ext4
---
apiVersion: storage.k8s.io/v1
kind: StorageClass
metadata:
    name: standard
provisioner: kubernets.io/gce-pd
parameters:
    type: pd-standard
    replication-type: node
---
apiVersion: storage.k8s.io/v1
kind: StorageClass
metadata:
    name: ssd
provisioner: kubernets.io/gce-pd
parameters:
    type: pd-ssd
    replication-type: regional-pd # high availability
    zones: us-central1-a, us-central1-b
```

accessModes:
- ReadWriteOnce # mount as read/write to one node
- ReadOnlyMany # mount as read only to many nodes
- ReadWriteMany # mount as read/write to many nodes, eg. NFS. But GCP persistent disks do not support ReadWriteMany

__PersistentVolumeClaim(PVC)__

- consume by pod

```
apiVersion: v1
kind: PersistentVolumeClaim
matadata:
    name: pd-volume-claim
spec:
    storageClassName: "standard"
    accessModes:
    - ReadWriteOnce
    resources:
        requests:
            storage: 100G
    # By default, deleting the persistent volume claim will also delete the provisioned persistent volume. 
    # If you want to retain the persistent volume, set its persistent volume reclaim policy to retain
    persistentVolumeReclaimPolicy: Retain 
```